In [1]:
import numpy as np
np.random.seed(7)
from scanner import *
from localizer import *
from tools import *
from models import *
from keras.callbacks import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using TensorFlow backend.


In [2]:
wifi = wifi_scanner('wlp3s0')

lab_loc = localizer(wifi)
lab_loc.make_profile('lab_profile_bigger')
#lab_loc.load_profile('lab_profile_bigger')
print(lab_loc)

# load and scale the data
X_train, y_train, X_test, y_test = load_data_from_folder('../datasets', lab_loc.profile, 
                                                         train_test_split=0.9, keep_percent=1.0, 
                                                         item='rssi')

shape_str = '{:10s} | {:15s} | {:15s}'
print(shape_str.format('', 'X shape', 'y shape'))
print(shape_str.format('TRAIN', str(X_train.shape), str(y_train.shape)))
print(shape_str.format('TEST', str(X_test.shape), str(y_test.shape)))

Creating new profile: lab_profile_bigger


----------------------------------------
Localizer
Wifi Device: wlp3s0
Profile: lab_profile_bigger Length: 57
Routers:
    260453955618305
    260453961510865
    260453955618306
    260453955618307
    260453955618310
    260453961510866
    260453961510867
    260453961510870
    260453955618304
    260453955571521
    260453955571520
    260453955571522
    260453955571523
    260453954156657
    260453954156662
    260453954156660
    260453954156656
    260453954156659
    73624541891
    13640875216131
    13640875216129
    13640875216134
    13640875216130
    260453961510545
    260453961510544
    13640875215889
    13640875215890
    13640875215891
    13640875215888
    260453961511585
    260453961511586
    260453961509698
    968197069745
    260453961509697
    260453955570576
    260453961511584
    968197069744
    260453955570578
    968197069750
    260453961511587
    968197069748
    260453961509699
    260453962890784
    968197069747
    260453955570579
    2604

2017-06-28_13:52 loaded


2017-07-03_15:32 loaded
2017-07-03_15:26 loaded


2017-07-05_14:57 loaded
Loaded 4 files from ../datasets
           | X shape         | y shape        
TRAIN      | (2070, 57)      | (2070, 2)      
TEST       | (230, 57)       | (230, 2)       


In [5]:
# build model
x_model = Sequential()
x_model.add(Dense(256, kernel_initializer='he_normal', input_shape=(len(lab_loc.profile),)))
x_model.add(Activation('relu'))
x_model.add(Dense(1024, kernel_initializer='he_normal'))
x_model.add(Activation('relu'))
x_model.add(Dense(256, kernel_initializer='he_normal'))
x_model.add(Activation('relu'))
x_model.add(Dense(1, kernel_initializer='he_normal'))
x_model.add(Activation('linear'))   

x_model.compile(optimizer='rmsprop', loss='mse')

# build model
y_model = Sequential()
y_model.add(Dense(256, kernel_initializer='he_normal', input_shape=(len(lab_loc.profile),)))
y_model.add(Activation('relu'))
y_model.add(Dense(1024, kernel_initializer='he_normal'))
y_model.add(Activation('relu'))
y_model.add(Dense(256, kernel_initializer='he_normal'))
y_model.add(Activation('relu'))
y_model.add(Dense(1, kernel_initializer='he_normal'))
y_model.add(Activation('linear'))   

y_model.compile(optimizer='rmsprop', loss='mse')


epochs = 10
batch_size = 32

x_model.fit(X_train, y_train[:,0], epochs=epochs, batch_size=batch_size, validation_split=0.1,
             verbose=0)
y_model.fit(X_train, y_train[:,1], epochs=epochs, batch_size=batch_size, validation_split=0.1,
             verbose=0)

In [5]:
print('X_model')
print('Training RMSE: {:.5f}'.format(np.sqrt(x_model.evaluate(X_train, y_train[:,0], verbose=0))))
print('Testing RMSE: {:.5f}\n'.format(np.sqrt(x_model.evaluate(X_test, y_test[:,0], verbose=0))))

print('Y_model')
print('Training RMSE: {:.5f}'.format(np.sqrt(y_model.evaluate(X_train, y_train[:,1], verbose=0))))
print('Testing RMSE: {:.5f}\n'.format(np.sqrt(y_model.evaluate(X_test, y_test[:,1], verbose=0))))

X_model
Training RMSE: 0.12035
Testing RMSE: 0.12791

Y_model


Training RMSE: 0.22739
Testing RMSE: 0.24095



In [13]:
# for testing a file as model input data
X_temp, y_temp = load_data_from_file('../datasets/old_datasets/2017-07-06_10:50', lab_loc.profile, 
                                     keep_percent=1.0, item='rssi')
print(X_temp)
print(y_model.predict(X_temp))

[[ 0.58571429  0.57142857  0.58571429 ...,  0.65714286  0.28571429
   0.41428571]
 [ 0.57142857  0.57142857  0.55714286 ...,  0.65714286  0.28571429
   0.41428571]
 [ 0.55714286  0.57142857  0.57142857 ...,  0.65714286  0.28571429
   0.41428571]
 ..., 
 [ 0.58571429  0.58571429  0.58571429 ...,  0.61428571  0.25714286
   0.44285714]
 [ 0.58571429  0.58571429  0.58571429 ...,  0.61428571  0.25714286
   0.44285714]
 [ 0.55714286  0.57142857  0.58571429 ...,  0.67142857  0.25714286
   0.41428571]]
[[ 1.08290327]
 [ 1.11297119]
 [ 1.04010117]
 [ 1.03063142]
 [ 1.05188823]
 [ 1.04801238]
 [ 0.95345336]
 [ 0.96238643]
 [ 0.9646911 ]
 [ 1.02571332]
 [ 1.03975928]
 [ 0.95567751]
 [ 0.9808163 ]
 [ 0.95872909]
 [ 0.96321744]
 [ 0.92971033]
 [ 0.96307158]
 [ 0.99472189]
 [ 0.90695971]
 [ 1.01532745]
 [ 0.96903968]
 [ 0.98168439]
 [ 0.88665289]
 [ 0.91949612]
 [ 0.9040333 ]
 [ 0.89397323]
 [ 0.9081071 ]
 [ 0.91721052]
 [ 0.88446897]
 [ 0.94161332]
 [ 0.92989343]
 [ 0.9521324 ]
 [ 0.92039073]
 [ 0.

In [14]:
for i in range(100):
    cells = np.expand_dims(lab_loc.wifi.get_wifi_cells(lab_loc.profile, item='rssi'), axis=0)
    print(cells)
    
    cells = scale_inputs(cells)
    print('X: ', x_model.predict(cells))
    print('Y: ', y_model.predict(cells))
    sleep(0.5)

[[ -58  -59  -59  -58  -62  -55  -62  -56  -61  -57  -56 -100 -100  -66
  -100  -57  -55  -55 -100  -70]]
X:  [[-0.0658101]]
Y:  [[ 1.33481276]]


[[ -58  -59  -58  -58  -62  -55  -62  -57  -61  -57  -57  -65  -65  -65
   -65  -55  -55  -55 -100  -73]]
X:  [[ 0.09439811]]
Y:  [[ 1.00913477]]


[[ -59  -59  -59  -59  -62  -57  -62  -57  -62  -56  -57  -65  -64  -64
   -65  -55  -55  -55 -100  -71]]
X:  [[ 0.06157761]]
Y:  [[ 1.08050668]]


[[ -58  -59  -58  -58  -62  -57  -62  -56  -62  -57  -57  -65  -65  -64
   -64  -55  -55  -55 -100  -71]]
X:  [[ 0.06453505]]
Y:  [[ 1.0342685]]


[[ -58  -59  -59  -59  -61  -57  -59  -56  -62  -57  -56  -65  -65  -64
   -64  -54  -55  -54 -100  -66]]
X:  [[ 0.00740764]]
Y:  [[ 1.21638572]]


...

[[-55 -55 -56 -55 -59 -57 -59 -58 -59 -57 -58 -73 -73 -72 -71 -59 -60 -57
  -82 -76]]
X:  [[ 0.11419354]]
Y:  [[ 0.89747179]]


[[-55 -55 -56 -55 -59 -57 -59 -58 -59 -57 -58 -73 -73 -72 -71 -59 -60 -57
  -85 -75]]
X:  [[ 0.11222515]]
Y:  [[ 0.95324159]]


[[-55 -55 -55 -55 -59 -57 -59 -57 -59 -57 -57 -73 -73 -73 -73 -59 -60 -59
  -85 -76]]
X:  [[ 0.167089]]
Y:  [[ 0.92785192]]


[[-55 -55 -55 -55 -59 -58 -59 -58 -59 -58 -58 -73 -72 -73 -73 -59 -59 -59
  -85 -75]]
X:  [[ 0.13603862]]
Y:  [[ 0.95680326]]


[[-55 -55 -55 -55 -59 -57 -59 -57 -58 -58 -58 -73 -73 -73 -73 -58 -58 -60
  -85 -77]]
X:  [[ 0.18898478]]
Y:  [[ 0.91129243]]
